In [2]:
import datetime
import os

import fiona
import geopandas as gpd
import geopy.distance
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange

fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'
storage = "/Volumes/easystore/Drones"
coi=['Incident No.','Description','cfs-date','Block Location','Latitude','Longitude','drone-id']

calls_for_service = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_calls-for-service-with-census-data.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

flight_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_drone-seconds-block-count-per-flight.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
cv_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/mapping/cv-blocks.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
cv_blocks['GEOID20'] = cv_blocks['GEOID20'].apply(lambda x: x[3:])
cv_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_cv-flight-manifest.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [3]:
calls_for_service["cfs-date"] = pd.to_datetime(calls_for_service["Date"])
cv_flights["drone-date"] =  pd.to_datetime(cv_flights['date'] + " " + cv_flights['time_s'])
cv_flights["match"] = cv_flights["incident_id"].astype(str).apply(lambda x: x.strip().replace(' ','').replace('L','0').replace('l','0')[-5:])
calls_for_service["match"] = (
    calls_for_service["Incident No."].astype(str).apply(lambda x: x.strip().replace(' ','').replace('l','0').replace('L','0')[-5:])
)

In [4]:
cv_flights[cv_flights['type'].astype(str).str.contains('ax')]

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,drone-date,match
2842,32.5944025,-117.0384457,3:52pm,ee6abec3c38cc6e321c6ecaae793f33a,2022-03-12,3:52pm,4380 Main,L020734,Brandish Weapon (axe),cvpd,2022-03-12 15:52:00,20734


In [5]:
matches = cv_flights[cv_flights["match"].isin(calls_for_service["match"])]
matches_joined_cfs = pd.merge(matches, calls_for_service, how="left", on="match",suffixes=['-drones','-cfs'])

matches_joined_cfs = matches_joined_cfs[
    matches_joined_cfs["drone-date"].dt.date == matches_joined_cfs["cfs-date"].dt.date
]
flights_with_cfs  = matches_joined_cfs.copy()



In [6]:
# 060730126001002
# 060730126001003
# 060730125012001

In [7]:
oaklawn = flight_blocks[(flight_blocks['GEOID20']=='060730126001002')|(flight_blocks['GEOID20']=='060730126001003')|
(flight_blocks['GEOID20']=='060730125012001')]
oaklawn = oaklawn[oaklawn['seconds'].astype(int)>60]
oaklawn_flight_ids = oaklawn['id'].drop_duplicates().values

oaklawn_flights = flights_with_cfs[flights_with_cfs['id'].isin(oaklawn_flight_ids)]
oaklawn_flights = oaklawn_flights[oaklawn_flights["drone-date"]>"2023-01-01"]
# oaklawn_flights[oaklawn_flights['type'].astype(str).str.contains('domestic')].values

In [9]:
oaklawn_flights[['type','Description','address_map','Address','Incident No.','incident_id','drone-date','cfs-date','id']].sort_values('cfs-date')

# possibly https://app.airdata.com/flight/24e7ced12192e0bb44cfe4a77dc9ed9f
#or https://app.airdata.com/flight/42315eefc26ee89ab56d1ba119c612de


,type,Description,address_map,Address,Incident No.,incident_id,drone-date,cfs-date,id
11427,Traffic Hazard,Traffic Hazard,Woodlawn Ave./ H St.,"400 BLOCK WOODLAWN AV, Chula Vista, CA, 91910",230108L00002096,CVL02096,2023-01-08 13:20:00,2023-01-08 13:21:36.747,45f0384e94de8de620cae6b44439ec92
11407,Welfare Check,Narcotics,745 H St.,"700 BLOCK H ST, Chula Vista, CA, 91910",230108L00002099,CVL02099,2023-01-08 13:39:00,2023-01-08 13:37:55.137,5a79fd74b2007c63d25a195f4c67cbbe
11634,Traffic Hazard,Disturbance - General,600 H St.,"600 BLOCK H ST, Chula Vista, CA, 91910",230115L00004163,CVL04163,2023-01-15 14:23:00,2023-01-15 13:31:03.387,fab0686444b09c6a13334d60cab32ebf
11755,Weapon Threat,Weapon Threat,600 H St.,"1000 BLOCK BROADWAY, Chula Vista, CA, 91911",230119L00005233,CVL05233,2023-01-19 14:00:00,2023-01-19 13:45:55.693,71d207e4c57637326cb7295789e19cfa
11888,Suspicious Vehicle,Suspicious Vehicle,500 Oaklawn Ave.,"500 BLOCK OAKLAWN AV, Chula Vista, CA, 91910",230122L00006029,CVL06029,2023-01-22 09:29:00,2023-01-22 09:30:13.253,bf86be1416da7149b0a8fd4e6e8b201d
12042,Restraining Order Violation,[Protected Call Type],394 Broadway,"300 BLOCK BROADWAY, Chula Vista, CA, 91910",230126L00007314,CVL07314,2023-01-26 10:36:00,2023-01-26 10:30:25.363,daf5b97b50efec88bba7460a44d6ea66
12187,Subject causing a disturbance,SRO Campus Visit Greet,510 Broadway,"1100 BLOCK CAMINO BISCAY, Chula Vista, CA, 91910",230131L00008819,CVL008819,2023-01-31 09:03:00,2023-01-31 09:00:52.067,2c122a9533740078b981d71561f9c87e
12371,Narcotics,Reckless Driver,Woodlawn Ave/ H St,"100 BLOCK BROADWAY, Chula Vista, CA, 91910",230208L00011429,CVL011429,2023-02-08 16:31:00,2023-02-08 12:34:13.703,3656f1fae59c85ebf1fd2e3603cc55ba
12491,Theft,Petty Theft <=$950,700 H Street,"700 BLOCK H ST, Chula Vista, CA, 91910",230212L00012688,cvl12688,2023-02-12 09:33:00,2023-02-12 09:43:19.080,5fe28e969e2b234955dce7250e41304a
12477,Welfare Check,[Protected Call Type],20 Fourth Ave.,"1 BLOCK FOURTH AV, Chula Vista, CA, 91910",230212L00012738,CVL12738,2023-02-12 14:31:00,2023-02-12 14:46:57.920,133fc6a7567ea31d0283867bdbf8a59f


In [31]:
# calls_for_service[((calls_for_service['cfs-date'])>"2023-08-28 15:00")&((calls_for_service['cfs-date'])<"2023-08-28 17:00")].sort_values('cfs-date')

kidnapping = calls_for_service[((calls_for_service['Description']).astype(str).str.contains('ax',case=False))]

In [32]:
kidnapping

,Incident No.,Date,Description,Priority,Disposition,Call Source,Zip Code,Block Location,Case No,Address,...,Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code,cfs-date,match


In [21]:
flights_with_cfs[flights_with_cfs['Incident No.'].isin(kidnapping['Incident No.'])]

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,...,Full FIPS (block),Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code,cfs-date
2628,32.6475513,-117.0886218,12:32pm,9e1e04dc632c8e374135346fd737545f,2021-09-01,12:32pm,400 CASSLEMAN,L073400,DOMESTIC VIOLENCE KIDNAPPING,cvpd,...,060730124012001,06073012401,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-09-01 12:41:46.917
4811,32.6178394,-117.091341,2:00pm,14b4abdadd2c0484c960f8419b899413,2022-02-03,2:00pm,I-5/ E St.,CVL9628,Robbery,cvpd,...,060730133082003,06073013308,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-02-03 12:32:23.447
4813,32.6178394,-117.091341,1:55pm,9aed4160cd8b8b65574e27068ff414ef,2022-02-03,1:55pm,I-5/ E St.,CVL9628,Robbery,cvpd,...,060730133082003,06073013308,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-02-03 12:32:23.447
4821,32.6569087,-117.0819565,12:27pm,50ae7b00753658c6fe7ec92259b26f34,2022-02-03,12:27pm,Second Ave./ E St.,CVL9628,Robbery,cvpd,...,060730133082003,06073013308,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-02-03 12:32:23.447
4823,32.6569087,-117.0819565,1:07pm,802e18217a1775a79a3291e61c33f87a,2022-02-03,1:07pm,Second Ave./ E St.,CVL9628,Robbery,cvpd,...,060730133082003,06073013308,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-02-03 12:32:23.447
5218,32.7340904,-117.1132733,9:47am,f5ef8491399c919663b41638ce8a8a1c,2022-02-17,9:47am,Bonita Rd./ I-805,CVL13875,Unknown Problem,cvpd,...,060730123042002,06073012304,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-02-17 09:14:06.237
5224,32.7340904,-117.1132733,9:16am,8939ef795f1c8ea1de2059f4e87a218c,2022-02-17,9:16am,Bonita Rd./ I-805,CVL13875,Unknown Problem,cvpd,...,060730123042002,06073012304,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-02-17 09:14:06.237
8776,32.5984543,-117.0645288,2:28pm,5d3548e312f02321f54cdbda363b264c,2022-07-14,2:28pm,Third Ave/ Anita St,CVL057252,Unknown problem,cvpd,...,060730132042000,06073013204,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-07-14 14:39:34.507
11100,32.6340298,-116.9790598,4:31pm,795c2840248816a499c1addf8b48cd57,2022-09-24,4:31pm,1200 Stampede Wy.,CVL79469,Kidnapping,cvpd,...,060730133231005,06073013323,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2022-09-24 16:09:15.743
14136,32.60443,-117.088395,5:37pm,68177f11d11714d24fdf46a9e122a4fd,2023-05-05,5:37pm,800 Palomar St.,CVL37794,Unknown Problem,cvpd,...,060730131031000,06073013103,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2023-05-05 17:35:50.333


In [45]:
hbs = flight_blocks[flight_blocks['GEOID20']=='060730131032011']
hbs = hbs[hbs['seconds'].astype(int)>60]
hbs_flight_ids = hbs['id'].drop_duplicates().values
hbs_flights = cv_flights[cv_flights['id'].isin(hbs_flight_ids)]

In [47]:
hbs_flights.to_csv('./hbs.csv')

In [25]:
compiled_flights =[] 

for _, row in tqdm(circle_k_flights.iterrows()):
    
    d = gpd.read_file(f"/Volumes/easystore/Drones/flights/kml/{row['id']}.kml",driver='KML')
    d['drone-id'] = row['id']
    d['address_map'] = row['address_map']
    d['drone-date'] = row['drone-date']
    d['incident_id'] = row['incident_id']
    d['type']=row['type']
     
    compiled_flights.append(d)

0it [00:00, ?it/s]

In [21]:
paths = pd.concat(compiled_flights,ignore_index=True)
paths = paths[paths['Name']!="Home"]

In [22]:
paths

,Name,Description,geometry,drone-id,address_map,drone-date,incident_id,type
1,Airdata.com,,"LINESTRING Z (-117.08271 32.63996 22.16603, -1...",1131f77a1092f5967e8067cf7832e4a0,70 BROADWAY,2021-07-08,L056623,PERSON CAUSING A DISTURBANCE
3,Airdata.com,,"LINESTRING Z (-117.08273 32.63997 22.14031, -1...",a7d280b615c5237588d131f4df492758,BROADWAY/D,2021-09-16,L078085,INDECENT EXPOSURE
5,Airdata.com,,"LINESTRING Z (-117.08271 32.63998 22.13929, -1...",4d269d06100c8aec57af2e340033b26c,Broadway/ D St.,2022-01-13,CVL3347,Suspicious Circumstances
7,Airdata.com,,"LINESTRING Z (-117.08272 32.64001 22.12669, -1...",2a96f210aca563eea759dec8f6ecfabb,100 Broadway,2022-05-24,CVL42354,Restraining Order Violation
9,Airdata.com,,"LINESTRING Z (-117.08271 32.63997 22.14589, -1...",8953163dcbd9fd8223d75faf23d15c7e,500 Flower St.,2022-07-02,CVL53795,Domestic Violence
11,Airdata.com,,"LINESTRING Z (-117.08271 32.63996 22.16603, -1...",08c2329038a9ff89e5ee71a3ef11c187,00 N. Broadway,2022-08-14,CVL66778,Suspicious Person
13,Airdata.com,,"LINESTRING Z (-117.08270 32.63997 22.14233, -1...",38a2e59f50d28f7275dc528b4c1ccf34,600 Chula Vista St.,2022-08-28,CVL71229,Domestic Violence
15,Airdata.com,,"LINESTRING Z (-117.08272 32.63997 22.15412, -1...",7633b7a61d3e0e3ab7ba9c712cc3a990,Broadway/ Casselman St,2022-09-12,CVL075951,Illegally parked vehicle
17,Airdata.com,,"LINESTRING Z (-117.08271 32.63996 22.16603, -1...",a36769aebb61d457ff9f485badb3b784,100 Broadway,2023-03-08,CVL020183,Domestic Violence
19,Airdata.com,,"LINESTRING Z (-117.08270 32.63998 22.13710, -1...",27e08be721844a25da566e0f5794846f,70 Broadway,2023-04-15,CVL31678,Suspicious Person
